In [ ]:

70from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### to split dataset file

In [ ]:
file_list = '/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/FileList.csv'

In [ ]:
import pandas as pd

df = pd.read_csv( '/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/FileList.csv')

In [ ]:
df.head()

,FileName,EF,ESV,EDV,FrameHeight,FrameWidth,FPS,NumberOfFrames,Split
0,0X100009310A3BD7FC,78.498406,14.881368,69.210534,112,112,50,174,VAL
1,0X1002E8FBACD08477,59.101988,40.383876,98.742884,112,112,50,215,TRAIN
2,0X1005D03EED19C65B,62.363798,14.267784,37.909734,112,112,50,104,TRAIN
3,0X10075961BC11C88E,54.545097,33.143084,72.914210,112,112,55,122,TRAIN
4,0X10094BA0A028EAC3,24.887742,127.581945,169.855024,112,112,52,207,VAL


### split with mixed test set

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Example DataFrame for demonstration
# df = pd.read_csv('your_file.csv')

# Define EF range for normal
min_ef = 53  # minimum EF value for normal
max_ef = 70  # maximum EF value for normal
validation_proportion = 0.1  # 10% of normal EF data for validation

# Step 1: Add 'EF_Status' column
df['EF_Status'] = df['EF'].apply(lambda x: 'N' if min_ef <= x <= max_ef else 'A')

# Step 2: Initialize 'Split' column
df['Split'] = 'UNASSIGNED'  # Start with all as 'UNASSIGNED'

# Step 3: Assign all abnormal samples to TEST temporarily
test_abnormal_df = df[df['EF_Status'] == 'A']

# Step 4: Split normal EF data into TRAIN_VAL (90%) and TEST_NORMAL (10%)
normal_ef_df = df[df['EF_Status'] == 'N']
train_val_df, test_normal_df = train_test_split(
    normal_ef_df,
    test_size=0.1,  # 10% to TEST
    random_state=42
)

# Step 5: Balance the TEST set
# Downsample the larger class in TEST
if len(test_normal_df) > len(test_abnormal_df):
    test_normal_df = test_normal_df.sample(n=len(test_abnormal_df), random_state=42)
else:
    test_abnormal_df = test_abnormal_df.sample(n=len(test_normal_df), random_state=42)

# Combine the balanced TEST set
balanced_test_df = pd.concat([test_normal_df, test_abnormal_df])

# Step 6: Assign splits
# Assign balanced TEST split
df.loc[balanced_test_df.index, 'Split'] = 'TEST'

# Assign remaining normal EF data to TRAIN_VAL
train_df, val_df = train_test_split(
    train_val_df,
    test_size=validation_proportion / (1 - 0.1),  # Adjusted proportion to get 10% of original normal EF data
    random_state=42
)

# Assign TRAIN and VAL splits
df.loc[train_df.index, 'Split'] = 'TRAIN'
df.loc[val_df.index, 'Split'] = 'VAL'

# Step 7: Verify the splits
print("Counts per Split:")
print(df['Split'].value_counts())
print("\nCounts per EF_Status:")
print(df['EF_Status'].value_counts())
print("\nEF_Status counts within each Split:")
print(df.groupby('Split')['EF_Status'].value_counts())

# Save the updated DataFrame
df.to_csv('/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/updated_file.csv', index=False)

Counts per Split:
Split
TRAIN         5616
UNASSIGNED    2305
TEST          1406
VAL            703
Name: count, dtype: int64

Counts per EF_Status:
EF_Status
N    7022
A    3008
Name: count, dtype: int64

EF_Status counts within each Split:
Split       EF_Status
TEST        A             703
            N             703
TRAIN       N            5616
UNASSIGNED  A            2305
VAL         N             703
Name: count, dtype: int64


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your DataFrame
# df = pd.read_csv('your_file.csv')

# Define EF range for normal
min_ef = 53  # minimum EF value for normal
max_ef = 70  # maximum EF value for normal
validation_proportion = 0.1  # 10% of normal EF data for validation

# Step 1: Add 'EF_Status' column
df['EF_Status'] = df['EF'].apply(lambda x: 'N' if min_ef <= x <= max_ef else 'A')

# Step 2: Initialize 'Split' column
df['Split'] = 'UNASSIGNED'  # Start with all as 'UNASSIGNED'

# Assign 'TEST' to all data outside the normal EF range (EF_Status == 'A')
df.loc[df['EF_Status'] == 'A', 'Split'] = 'TEST'

# Step 3: Handle Data Within the Normal EF Range
normal_ef_df = df[df['EF_Status'] == 'N']

# Split normal EF data into TRAIN_VAL (90%) and TEST_NORMAL (10%)
train_val_df, test_normal_df = train_test_split(
    normal_ef_df,
    test_size=0.1,  # 10% to TEST
    random_state=42
)

# Split TRAIN_VAL into TRAIN and VAL
train_df, val_df = train_test_split(
    train_val_df,
    test_size=validation_proportion / (1 - 0.1),  # Adjusted proportion to get 10% of original normal EF data
    random_state=42
)

# Step 4: Update 'Split' column
df.loc[train_df.index, 'Split'] = 'TRAIN'
df.loc[val_df.index, 'Split'] = 'VAL'
df.loc[test_normal_df.index, 'Split'] = 'TEST'

# Save the updated DataFrame
df.to_csv('/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/updated_file.csv', index=False)

# Display the updated DataFrame
print(df.head())

# Optional: Check the counts
print("Counts per Split:")
print(df['Split'].value_counts())
print("\nCounts per EF_Status:")
print(df['EF_Status'].value_counts())

# Optional: Check the counts of EF_Status in each split
print("\nEF_Status counts within each Split:")
print(df.groupby('Split')['EF_Status'].value_counts())


             FileName         EF         ESV         EDV  FrameHeight  \
0  0X100009310A3BD7FC  78.498406   14.881368   69.210534          112   
1  0X1002E8FBACD08477  59.101988   40.383876   98.742884          112   
2  0X1005D03EED19C65B  62.363798   14.267784   37.909734          112   
3  0X10075961BC11C88E  54.545097   33.143084   72.914210          112   
4  0X10094BA0A028EAC3  24.887742  127.581945  169.855024          112   

   FrameWidth  FPS  NumberOfFrames  Split EF_Status  
0         112   50             174   TEST         A  
1         112   50             215    VAL         N  
2         112   50             104    VAL         N  
3         112   55             122  TRAIN         N  
4         112   52             207   TEST         A  
Counts per Split:
Split
TRAIN    5616
TEST     3711
VAL       703
Name: count, dtype: int64

Counts per EF_Status:
EF_Status
N    7022
A    3008
Name: count, dtype: int64

EF_Status counts within each Split:
Split  EF_Status
TEST   A    

### First split for training

In [ ]:
# Define EF range for training
min_ef = 53  # minimum EF value for training
max_ef = 70  # maximum EF value for training

# Define validation split proportion
validation_proportion = 0.1  # 10% of training set for validation

# Step 1: Classify each row as 'TR' (train) or 'TEST' based on EF range
df['Split'] = df['EF'].apply(lambda x: 'TRAIN' if min_ef <= x <= max_ef else 'TEST')

# Step 2: Filter the training set
training_df = df[df['Split'] == 'TRAIN']

# Step 3: Sample a portion of the training set for validation
validation_size = int(len(training_df) * validation_proportion)
validation_df = training_df.sample(n=validation_size, random_state=42)

# Step 4: Update the 'Split' column for validation samples
df.loc[validation_df.index, 'Split'] = 'VAL'

# Save the updated DataFrame if needed
df.to_csv('/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/updated_file.csv', index=False)

# Display the updated DataFrame
print(df.head())

             FileName         EF         ESV         EDV  FrameHeight  \
0  0X100009310A3BD7FC  78.498406   14.881368   69.210534          112   
1  0X1002E8FBACD08477  59.101988   40.383876   98.742884          112   
2  0X1005D03EED19C65B  62.363798   14.267784   37.909734          112   
3  0X10075961BC11C88E  54.545097   33.143084   72.914210          112   
4  0X10094BA0A028EAC3  24.887742  127.581945  169.855024          112   

   FrameWidth  FPS  NumberOfFrames  Split  
0         112   50             174   TEST  
1         112   50             215  TRAIN  
2         112   50             104  TRAIN  
3         112   55             122  TRAIN  
4         112   52             207   TEST  


In [ ]:
split_counts = df['Split'].value_counts()
split_counts

,count
Split,
TRAIN,6320
TEST,3008
VAL,702


In [ ]:
source_folder = '/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/'
destination_base_folder = '/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/'

In [ ]:
# Define maximum number of cases for each set
max_train_cases = 300
max_val_cases = 100
max_test_cases = 50

In [ ]:
# Step 1: Limit each set to the specified number of cases
train_df = df[df['Split'] == 'TRAIN'].sample(n=max_train_cases, random_state=42) if len(df[df['Split'] == 'TRAIN']) > max_train_cases else df[df['Split'] == 'TRAIN']
val_df = df[df['Split'] == 'VAL'].sample(n=max_val_cases, random_state=42) if len(df[df['Split'] == 'VAL']) > max_val_cases else df[df['Split'] == 'VAL']
test_df = df[df['Split'] == 'TEST'].sample(n=max_test_cases, random_state=42) if len(df[df['Split'] == 'TEST']) > max_test_cases else df[df['Split'] == 'TEST']

# Combine the limited sets into a single DataFrame for processing
final_df = pd.concat([train_df, val_df, test_df])
import os
import re

### rename th files back to original

In [ ]:
import os
import re
# Regular expression to match ' (1)', ' (2)', etc. at the end of the filename before the extension
pattern = r" \(\d+\)(?=\.\w+$)"

# Loop through each file in the folder
for filename in os.listdir(source_folder):
    # Check if the filename contains the pattern
    if re.search(pattern, filename):
        # Define the new filename by removing the suffix
        new_filename = re.sub(pattern, "", filename)
        # Rename the file
        os.rename(os.path.join(source_folder, filename), os.path.join(source_folder, new_filename))
        print(f"Renamed '{filename}' to '{new_filename}'")

In [ ]:
import os

# Loop through each file in the folder
for filename in os.listdir(source_folder):
    if filename.startswith("Copy of "):
        # Define the new filename by removing the "Copy of " prefix
        new_filename = filename.replace("Copy of ", "", 1)
        # Rename the file
        os.rename(os.path.join(source_folder, filename), os.path.join(source_folder, new_filename))
        print(f"Renamed '{filename}' to '{new_filename}'")

Renamed 'Copy of 0X1A0A263B22CCD966 (2).avi' to '0X1A0A263B22CCD966 (2).avi'
Renamed 'Copy of 0X1A2E9496910EFF5B (2).avi' to '0X1A2E9496910EFF5B (2).avi'
Renamed 'Copy of 0X1A2C60147AF9FDAE (2).avi' to '0X1A2C60147AF9FDAE (2).avi'
Renamed 'Copy of 0X1A6ACFE7B286DAFC (2).avi' to '0X1A6ACFE7B286DAFC (2).avi'
Renamed 'Copy of 0X1A3E7BF1DFB132FB (2).avi' to '0X1A3E7BF1DFB132FB (2).avi'
Renamed 'Copy of 0X1A8D85542DBE8204 (2).avi' to '0X1A8D85542DBE8204 (2).avi'
Renamed 'Copy of 0X1A2A76BDB5B98BED (2).avi' to '0X1A2A76BDB5B98BED (2).avi'
Renamed 'Copy of 0X1A3D565B371DC573 (2).avi' to '0X1A3D565B371DC573 (2).avi'
Renamed 'Copy of 0X1A05DFFFCAFB253B (2).avi' to '0X1A05DFFFCAFB253B (2).avi'
Renamed 'Copy of 0X1A5FAE3F9D37794E (2).avi' to '0X1A5FAE3F9D37794E (2).avi'
Renamed 'Copy of 0X1A030EFDD45062FA (2).avi' to '0X1A030EFDD45062FA (2).avi'
Renamed 'Copy of 0X1A76A1A8448B456 (2).avi' to '0X1A76A1A8448B456 (2).avi'
Renamed 'Copy of 0X1A9D7251E9464D49 (2).avi' to '0X1A9D7251E9464D49 (2).avi'
R

### perform thedatset split specify numbers.

In [ ]:
import os
import shutil
import pandas as pd

# Paths and settings
base_path = '/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/'  # replace with your actual folder path
csv_path = '/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/updated_file.csv'  # replace with your CSV file path

# Define the specific sample size for each tag
sample_sizes = {
    'TRAIN': 300,  # Number of files for training
    'TEST': 2000,   # Number of files for testing
    'VAL': 100     # Number of files for validation
}

# Step 1: Clear existing folders and recreate them
for set_name in sample_sizes.keys():
    folder_path = os.path.join(base_path, set_name)
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)  # Deletes the folder and all its contents
    os.makedirs(folder_path)  # Recreate the empty folder

# Step 2: Load the CSV file
df = pd.read_csv(csv_path)

# Step 3: Process each tag with specified sample sizes
for tag, sample_size in sample_sizes.items():
    # Filter files with the specific tag
    tagged_files = df[df['Split'] == tag]

    # Sample the specified number of files (if fewer than sample_size exist, it will take all available)
    sampled_files = tagged_files.sample(n=min(sample_size, len(tagged_files)), random_state=42)

    # Copy each sampled file to the respective folder
    for _, row in sampled_files.iterrows():
        filename = row['FileName'].strip()  # Remove any extra spaces
        if not filename.endswith('.avi'):
            filename += '.avi'  # Add the .avi extension if missing

        src_path = os.path.join(base_path, filename)
        dest_path = os.path.join(base_path, tag, filename)

        # Debugging: print the paths
        print(f"Attempting to copy: {src_path} to {dest_path}")

        # Check if file exists and copy
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_path)
        else:
            print(f"File not found: {src_path}")

# Step 4: Output summary
print("File distribution across splits:")
print(df['Split'].value_counts())

Attempting to copy: /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/0X6E00FB047E13A652.avi to /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/TRAIN/0X6E00FB047E13A652.avi
Attempting to copy: /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/0X47531256525CE22B.avi to /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/TRAIN/0X47531256525CE22B.avi
Attempting to copy: /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/0X11BE656DE80C9CF2.avi to /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/TRAIN/0X11BE656DE80C9CF2.avi
Attempting to copy: /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/0X3A58A66B6AF3DE55.avi to /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/TRAIN/0X3A58A66B6AF3DE55.avi
Attempting to copy: /content/drive/MyDrive/kaggle_data/input

In [ ]:
import os
import shutil

# Paths and settings
base_path = '/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/'  # replace with your actual folder path
csv_path = '/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/updated_file.csv'  # replace with your CSV file path

# Define the specific sample size for each tag
sample_sizes = {
    'TRAIN': 300,  # Number of files for training
    'TEST': 2000,   # Number of files for testing
    'VAL': 100     # Number of files for validation
}

# Create destination folders if they don't exist
for set_name in sample_sizes.keys():
    os.makedirs(os.path.join(base_path, set_name), exist_ok=True)

# Load the CSV file
df = pd.read_csv(csv_path)

# Process each tag with specified sample sizes
for tag, sample_size in sample_sizes.items():
    # Filter files with the specific tag
    tagged_files = df[df['Split'] == tag]

    # Sample the specified number of files (if fewer than sample_size exist, it will take all available)
    sampled_files = tagged_files.sample(n=min(sample_size, len(tagged_files)), random_state=42)

# Copy each sampled file to the respective folder
    for _, row in sampled_files.iterrows():
        filename = row['FileName'].strip()  # Remove any extra spaces
        if not filename.endswith('.avi'):
            filename += '.avi'  # Add the .avi extension if missing

        src_path = os.path.join(base_path, filename)
        dest_path = os.path.join(base_path, tag, filename)

        # Debugging: print the paths
        print(f"Attempting to copy: {src_path} to {dest_path}")

        # Check if file exists and copy
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_path)
        else:
            print(f"File not found: {src_path}")

Attempting to copy: /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/0X6E00FB047E13A652.avi to /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/TRAIN/0X6E00FB047E13A652.avi
Attempting to copy: /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/0X47531256525CE22B.avi to /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/TRAIN/0X47531256525CE22B.avi
Attempting to copy: /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/0X11BE656DE80C9CF2.avi to /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/TRAIN/0X11BE656DE80C9CF2.avi
Attempting to copy: /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/0X3A58A66B6AF3DE55.avi to /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/TRAIN/0X3A58A66B6AF3DE55.avi
Attempting to copy: /content/drive/MyDrive/kaggle_data/input

### full dataset split

In [ ]:
import os
import shutil
import pandas as pd

# Paths and settings
video_base_path = '/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/'  # Path to the video files
split_base_path = '/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/full_data/'  # Path to place split folders
csv_path = '/content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/updated_file.csv'  # Path to your CSV file

# Load the CSV file
df = pd.read_csv(csv_path)

# Get unique splits from the 'Split' column
unique_splits = df['Split'].unique()

# Create destination folders for each split in the split_base_path
for split in unique_splits:
    os.makedirs(os.path.join(split_base_path, split), exist_ok=True)

# Process each unique split
for split in unique_splits:
    # Filter files belonging to the current split
    split_files = df[df['Split'] == split]

    # Copy each file to the respective split folder
    for _, row in split_files.iterrows():
        filename = row['FileName'].strip()  # Remove any extra spaces
        if not filename.endswith('.avi'):
            filename += '.avi'  # Add the .avi extension if missing

        src_path = os.path.join(video_base_path, filename)
        dest_path = os.path.join(split_base_path, split, filename)

        # Debugging: print the paths
        print(f"Attempting to copy: {src_path} to {dest_path}")

        # Check if file exists and copy
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_path)
        else:
            print(f"File not found: {src_path}")

Streaming output truncated to the last 5000 lines.
Attempting to copy: /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/0X36E57BE2B11E4E3E.avi to /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/full_data/TRAIN/0X36E57BE2B11E4E3E.avi
Attempting to copy: /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/0X36E731545F69ECDC.avi to /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/full_data/TRAIN/0X36E731545F69ECDC.avi
Attempting to copy: /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/0X36F52C92187D7452.avi to /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/full_data/TRAIN/0X36F52C92187D7452.avi
Attempting to copy: /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/Videos/0X36FB42EDBD9C3623.avi to /content/drive/MyDrive/kaggle_data/input/echonet-dynamic/EchoNet-Dynamic/full_data/TRAIN/0X36FB42EDBD9C3623.a